Data structures, mappings and the iterator pattern
==================================================

:hourglass: 3h

**Outline**
1. Container
2. Iterator pattern
3. Mapping
4. Other data structures

## 1. Container


### Typology

:hourglass: 45 min

> :question: Define the following terms: 
container: data structure that can hold data
ordered :  the way you add data to the data structure is important (not about how it is sorted)
mutable: can be altered
iterable: can be iterated over
sizable: you can ask the size of a data structure
allow duplicate : a container that allows for duplicates, ex. set does not (opposite duplicate-free)

> :question: associate the tags of the previous question to the following structures: 
collection: container, sizeable. The others are not known
sequence: collection, ordered; sizeable (no mention of mutability)
tuple: all of the above but immutable
list: all of the above
set: same a frozen set but mutable
frozen set: immutable, duplicate-free, container, iterable, sizeable

Tuple and list are sequences.
All of them are collections

you can have a container that is not an iterable, and you can have an iterable that is not a container. A container is sizeable

To have a container you need __contains__
to be sizeable you need __len__
to be iterable you need __iter__

### Interface (abstract data types) vs implementation
:hourglass: 15 min

An interface defines **what** a data structure can do. In Object Oriented Programming (OOP) terms, the interface defines the list of messages that an instance can publicly receive.

In order to be of use, a concrete implementation of the interface must be written. This implementation defines the **how**. 

| Interface | Implementations                          |
|-----------|------------------------------------------|
| List      | Linked-list, vector (ie. resiable array) |
| Set       | Hash table, linked-list, vector, etc.    |

The implementation determines the speed of an operation. For instance, if you implement a set with a linked list, some operations will be very slow. See the example below.

In [ ]:
def intersection(iterable, container):
    result = set()
    for x in iterable:
        if x in container:
            result.add(x)
    
    return result

import random

K = 1000000
N = 100000

a = [random.randint(0, K) for _ in range(N)]
b = [random.randint(0, K) for _ in range(N)]

In [ ]:
_ = intersection(a, b)

In [ ]:
_ = intersection(a, set(b))

Although the implementation determines the speed, you can expect a standard library to provide fast implementation of the usual data structure. 

> :question: in terms of "(almost) instantaneous", "fast" and "slow", what is the speed of the insertion and search operations for a list and a set?

#### Complexity :skull: 


In Computer Science, it is traditional to discuss speed in abstract terms, rather than in concrete time. This removes the hardware (and load) dependency (but introduces some hypotheses regarding the uniformity of some basic operations). For data structure, the *complexity* is established with respect to the size of the problem; the number of elements in the structure. 

If $N$ is the number of elements, we usually talk about $O(\log N)$, $O(N)$, $O(N \log N)$, $O(N^2)$. This notation informs on the behavior of an operation when $N$ is large. For instance, $O(N)$ means that, once $N$ is large enough, the operation takes a time proportional to $N$. So if you double $N$, you can expect to wait twice as long. $O(1)$ means that the runtime is independent of $N$.

Here are the typical complexities:
| Container | Implementation               | Operation | Complexity  |
|-----------|------------------------------|-----------|-------------|
| List      | Linked-list                  | Insert    | $O(1)$      |
| List      | Linked-list                  | Search    | $O(N)$      |
| Set       | Balanced binary sear tree    | Insert    | $O(\log N)$ |
| Set       | Balanced binary sear tree    | Search    | $O(\log N)$ |
| Set       | Hash table                   | Insert    | $O(1)$ (*)  |
| Set       | Hash table                   | Search    | $O(1)$ (*)  |

(*) amortized cost with a good hash function and load factor (or rehashing).

:coffee: 15 min

### Implementing a collection

:hourglass: 15 min

In Python, a collection must implement the `__len__` method, as well as the `__contains__`. The former is used via the `len` built-in function, and the latter via the `in` keyword.

In [ ]:
from typing import Any

    
class Collection:
    def __init__(self, *elements: Any) -> None:
        self._elements = elements

    def __len__(self) -> int:
        return len(self._elements)
    
    def __contains__(self, key: Any) -> bool:
        for x in self._elements:
            if x == key:
                return True
        return False

In [ ]:
collection = Collection(1, 2, 3)

print("length:", len(collection))
print("contains 2:", 2 in collection)
print("contains 5:", 5 in collection)

#### Equality :skull:

In the `__contains__` method we tested the equality of `x` and `key` via the `==` operator. 

:question: But can you predict the outcome of the cell below?

In [ ]:
from dataclasses import dataclass

class Person:
    def __init__(self, name: str) -> None:
        self._name = name

@dataclass
class Pokemon:
    name: str

p1 = Person("John")
p2 = Person("John")

print("p1 == p2", p1 == p2)
print("p1 == p1", p1 == p1)
print("p1 is p1", p1 is p1)

best_pk1 = Pokemon("Charmander")
best_pk2 = Pokemon("Charmander")

print("best_pk1 == best_pk2", best_pk1 == best_pk2)
print("best_pk1 == best_pk1", best_pk1 == best_pk1)
print("best_pk1 is p1", best_pk1 is best_pk1)

For custom objects, the default behavior is that `==` will test whether the left operand has the same memory address as the right operand (which is what the `is` keyword is doing). It is possible to override that behavior (eg. dataclass, there they check for the value, and this causes the difference in behavior). Note that there are a few principles to follow:
- if two objects are equivalent and hashable (define a `__hash__` method), the hash must be the same;
- you usually want two perfect clones (cf. `copy`) to be equivalent.



## 2. Iterator pattern


### Exposition
:hourglass: 15 min

One of the most common uses for containers, whatever the type, is to go over each element. It is common to expose a generic way to do that: the iterator pattern. 

In Python, the iterator *pattern* is built in the iterator *protocol* and it is what allows to use a for-loop to go over any container.

In [ ]:
def my_terribly_inefficient_lenght_function(container):
    n = 0
    for _ in container:
        n += 1
    return n

print("Length of list:", my_terribly_inefficient_lenght_function([1, 2, 3]))
print("Length of tuple:", my_terribly_inefficient_lenght_function((1, 2, 3)))
print("Length of set:", my_terribly_inefficient_lenght_function({1, 2, 3}))
print("Length of range:", my_terribly_inefficient_lenght_function(range(3)))


In Python, an *iterable* is any object having a `__iter__` method. The `__iter__` method returns an *iterator*. The iterator is the object that can be iterated upon. 

> As part of the iterator protocol, an iterator is iterable; it returns itself.

There are two ways to define an iterator:
- implement the full iterator protocol (define the `__iter__` and `__next__` methods; raise a `StopIteration` when there is no more items);
- use the `yield` (which creates a generator, which is an iterator).

Here is a (dummy) example of using the `yield` keyword:

In [ ]:
from typing import Any, Iterator

class MyList:
    def __init__(self, *elements: Any) -> None:
        self._elements = list(elements)

    def __iter__(self) -> Iterator:
        for x in self._elements:
            yield x

for x in MyList(2, 4, 6):
    print(x)

> The `__iter__` method can be invoked directly by using the `iter` built-in function on the object. Just as the `__len__` method is invoked by the `len` built-in function.

In [ ]:
iterator = iter(MyList(1, 2, 3))
print(iterator)
for x in iterator:
    print(x)

### Exercise

:hourglass: 30 min

Turn the following snippet of linked-list into a proper collection and implement the iterator protocol.


In [50]:
from typing import Optional, TypeVar, Generic, Iterator

T = TypeVar('T')

class Node(Generic[T]):
    def __init__(self, data: T) -> None:
        self.data = data
        self.next: Optional[Node[T]] = None

class LinkedList(Generic[T]):
    def __init__(self) -> None:
        self.head: Node[T] = None

    def append(self, data: T) -> None:
        new_node = Node(data)

        if not self.head:
            self.head = new_node
            return
        
        last_node = self.head
        while last_node.next:
            last_node = last_node.next
        last_node.next = new_node
        
            
    def __len__(self) -> int:
        n=1
        if not self.head:
            return 0
        node = self.head
        while node.next:
            n += 1
            node = node.next
        return n
    
    def __contains__(self, key: T) -> bool:
        for data in self:
	        if data == key:
		        return True
        return False

    def __iter__(self) -> Iterator:
        if not self.head:
            return

        node = self.head
        while node:
            yield node.data
            node = node.next


lijst = LinkedList()
lijst.append("een")
lijst.append(2)
lijst.append(3.0)

print(len(lijst))
assert(2 in lijst)

3


## 3. Mapping

:hourglass: 20 min

A mapping is a structure that *maps* a **key** to **value**. The most common mapping in Python is the dictionary. To implement a custom mapping you must redefine the following methods (see https://docs.python.org/3.9/library/collections.abc.html#collections-abstract-base-classes):
- `__getitem__`;
- `__setitem__`;
- `__delitem__`;
- `__iter__`;
- `__len__`.

> Note that a mapping cannot contain duplicate keys: it overwrites the associated value

In [ ]:
from collections.abc import MutableMapping
from typing import Iterator, NamedTuple, Any, Optional

class KVPair(NamedTuple):
    key: Any
    value: Any

class MyMapping(MutableMapping):
    def __init__(self, **kwargs) -> None:
        super().__init__()
        self._mapping = [KVPair(k, v) for k, v in kwargs.items()]

    def __getitem__(self, __key: Any) -> Any:
        print(f"Looking for key {__key!r}")
        for key, value in self._mapping:
            if key == __key:
                return value
        raise KeyError(__key)
    
    def _get_index_of(self, __key: Any) -> Optional[int]:
        for i, (key, _) in enumerate(self._mapping):
            if key == __key:
                return i
    
    def __setitem__(self, __key: Any, __value: Any) -> None:
        print(f"Associated key {__key!r} to value {__value!r} ")
    
        # Checking for replacement
        index: Optional[int] = self._get_index_of(__key)
        
        kvp = KVPair(__key, __value)
        if index is None:
            # Not found, append
            self._mapping.append(kvp)
        else:
            # Found at position `index`
            self._mapping[index] = kvp

    def __delitem__(self, __key: Any) -> None:
        print(f"Deleting key {__key!r} ")
        index: Optional[int] = self._get_index_of(__key)
        if index is not None:
            del self._mapping[index]

    def __iter__(self) -> Iterator:
        return iter(key for key, _ in self._mapping)
    
    def __len__(self) -> int:
        return len(self._mapping)


map_ = MyMapping()
print("map_", list(map_))  # Works thanks to the iterator protocol
map_["k1"] = 1
map_["k2"] = "Mewtwo"
map_["k1"] = 2

print("map_['k1']", map_['k1'])
print("map_", list(map_))

del map_["k1"]
print("map_", list(map_))

> In this case, by inheriting from `MutableMapping` we get additional methods (eg. `items`, `__contains__`). However, if it makes sense for the class you are designing, you can implement whichever method you want, and get the benefit of the `[]` syntax.

> Note how we implemented a mapping thanks to a container (admitedly, in an inefficient way). It is also straightforward to implement a set based on a dictionary: just use a dummy value (like `None`) as values.

### Custom objects as keys :skull:

In general, it is not a good idea to use a custom class as key for a dictionary:

In [ ]:
class MyClass:
    def __init__(self, i: int) -> None:
        self.i = i

{MyClass(2): 2}[MyClass(2)]

To use a custom class as key, you need to take the following precautions:
- make the class immutable;
- re-implement the `__eq__` and `__hash__` properly.

In general, two objects are equivalent if all their attributes are equivalent. A good hash should take into account all the attributes. Two equivalent objects must always have the same hash.

The immutability restriction follows from the above comment: if the object changes, its hash changes and therefore the mapping is corrupted.

> For non-hash-based mappings, the hash part is irrelevant but you would likely need to implement the comparison operators.

## 4. Closing words

:hourglass: 15 min

### Comprehension

Data structures and mappings are so ubiquitous that Python provides syntactic sugar to create the basic types.

In [ ]:
# List comprehension
[i**2 for i in range(10) if i % 3 == 0]

In [ ]:
# Set
{i**2 for i in range(10) if i % 3 == 0}

In [ ]:
# Dictionary
{i:i**2 for i in range(10) if i % 3 == 0}

In [ ]:
# Generator
(i**2 for i in range(10) if i % 3 == 0)

### Other data structures

In this modules we have discussed the most common data structure in Python (list, tuple, set, and dictionary), but there is a myriad of other structures:
- Stack (LIFO)
- Queue (FIFO), dequeue
- Frozenset
- Priority queues and heaps
- Trees containers (binary/non-binary, full, etc.)
- Tree dictionaries (binary search tree, AVL, red-black trees, etc.)
- Graphs
- defaultdict

See also for more https://docs.python.org/3.9/library/collections.abc.html#collections-abstract-base-classes

### What we have seen

In this module, we have discussed the different properties of data structures and dictionaries, as well as how to implement them and use the built-in iterator protocol of Python.

One of the goals of this module was to reflect on which data structure to use when and why.

**Dunderscore**:
- `__len__`
- `__contains__` 
- `__iter__`
- `__getitem__`
- `__setitem__`
- `__delitem__`